In [28]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import os as os
from selenium import webdriver
import sys
import time
from webdriver_manager.chrome import ChromeDriverManager

# Scrape MLPS neighborhood links 

In [29]:
# URL of page to be scraped
url = 'http://www.mncompass.org/profiles/neighborhoods/minneapolis-saint-paul'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [30]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('ul', class_='minneapolis-neighborhoods-listing')

#Declare lists to store neighborhood names and links
neighborhood=[]
neighborhood_link=[]

# Loop through returned results
for result in results:
    # Find all neighborhood data in a tags
    a_neighborhood = result.find_all('a')  
        
    # Error handling
    try:
        #Loop through the neighborhood a tags to extract neighborhood names and website links 
        for n in a_neighborhood:
        
        # Get name of neighborhood
            neighborhood_name=n.text
        
        # Identify and return link to neighborhood
            link = n['href']

        # Run only if neighborhood_name and link are available
            if (neighborhood_name and link):
                #Append to lists
                neighborhood.append(neighborhood_name)
                neighborhood_link.append(link)               

    except Exception as e:
        print(e)

In [31]:
#Create nighborhood household income dataframe
neighborhood_income_df=pd.DataFrame({"Neighborhood":neighborhood
                                    ,"Website_Link":neighborhood_link
                                    ,"IncomeLess35000_count":""
                                    ,"IncomeLess35000_percent":""
                                    ,"IncomeLess35to49_count":""
                                    ,"IncomeLess35to49_percent":""
                                    ,"IncomeLess50to74_count":""
                                    ,"IncomeLess50to74_percent":""
                                    ,"IncomeLess75to99_count":""
                                    ,"IncomeLess75to99_percent":""
                                    ,"Income100Plus_count":""
                                    ,"Income100Plus_percent":""
                                    ,"Median_Income_Total":""
                                  })

neighborhood_income_df.head()

,Neighborhood,Website_Link,IncomeLess35000_count,IncomeLess35000_percent,IncomeLess35to49_count,IncomeLess35to49_percent,IncomeLess50to74_count,IncomeLess50to74_percent,IncomeLess75to99_count,IncomeLess75to99_percent,Income100Plus_count,Income100Plus_percent,Median_Income_Total
0,Armatage,minneapolis/armatage,,,,,,,,,,,
1,Audubon Park,minneapolis/audubon-park,,,,,,,,,,,
2,Bancroft,minneapolis/bancroft,,,,,,,,,,,
3,Cedar - Isles - Dean,minneapolis/cedar-isles-dean,,,,,,,,,,,
4,Cedar Riverside,minneapolis/cedar-riverside,,,,,,,,,,,


In [32]:
#Build the neighborhood website link
neighborhood_income_df['Website_Link']= 'http://www.mncompass.org/profiles/neighborhoods/'+neighborhood_income_df['Website_Link']
neighborhood_income_df.head()

,Neighborhood,Website_Link,IncomeLess35000_count,IncomeLess35000_percent,IncomeLess35to49_count,IncomeLess35to49_percent,IncomeLess50to74_count,IncomeLess50to74_percent,IncomeLess75to99_count,IncomeLess75to99_percent,Income100Plus_count,Income100Plus_percent,Median_Income_Total
0,Armatage,http://www.mncompass.org/profiles/neighborhood...,,,,,,,,,,,
1,Audubon Park,http://www.mncompass.org/profiles/neighborhood...,,,,,,,,,,,
2,Bancroft,http://www.mncompass.org/profiles/neighborhood...,,,,,,,,,,,
3,Cedar - Isles - Dean,http://www.mncompass.org/profiles/neighborhood...,,,,,,,,,,,
4,Cedar Riverside,http://www.mncompass.org/profiles/neighborhood...,,,,,,,,,,,


# Scrape Household Income for neighborhoods in MLPS

In [33]:
#Run loop to traverse through city_weather_df and populate required fields in df
print("Beginning Data Retrieval")    
print("-----------------------------")

for index, row in neighborhood_income_df.iterrows():
    
    #launch url
    url = row['Website_Link']
    
    try:
    
        # create a new Chrome session
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        driver = webdriver.Chrome('C:/path/to/chromedriver.exe',options=options)
        driver.implicitly_wait(30)
        driver.get(url)

        #After opening the url above, Selenium clicks the specific income link
        python_button = driver.find_element_by_id('householdIncomeLess35000') 
    
        #click above link
        python_button.click() 
    
        #Selenium hands the page source to Beautiful Soup
        soup_level1=bs(driver.page_source, 'lxml')
    
    
        print(f"Processing Record {row['Neighborhood']}")
        print(f"URL: {row['Website_Link']}")
        
        neighborhood_income_df.loc[index, 'IncomeLess35000_count'] = soup_level1.find('td', class_='count', id='householdIncomeLess35000').text
        neighborhood_income_df.loc[index, 'IncomeLess35000_percent'] = soup_level1.find('td', class_='percent', id='householdIncomeLess35000').text
        neighborhood_income_df.loc[index, 'IncomeLess35to49_count'] = soup_level1.find('td', class_='count', id='householdIncomeLess35to49').text
        neighborhood_income_df.loc[index, 'IncomeLess35to49_percent'] = soup_level1.find('td', class_='percent', id='householdIncomeLess35to49').text
        neighborhood_income_df.loc[index, 'IncomeLess50to74_count'] = soup_level1.find('td', class_='count', id='householdIncomeLess50to74').text
        neighborhood_income_df.loc[index, 'IncomeLess50to74_percent'] = soup_level1.find('td', class_='percent', id='householdIncomeLess50to74').text
        neighborhood_income_df.loc[index, 'IncomeLess75to99_count'] = soup_level1.find('td', class_='count', id='householdIncomeLess75to99').text
        neighborhood_income_df.loc[index, 'IncomeLess75to99_percent'] = soup_level1.find('td', class_='count', id='householdIncomeLess75to99').text
        neighborhood_income_df.loc[index, 'Income100Plus_count'] = soup_level1.find('td', class_='count', id='householdIncome100Plus').text
        neighborhood_income_df.loc[index, 'Income100Plus_percent'] = soup_level1.find('td', class_='percent', id='householdIncome100Plus').text        
        neighborhood_income_df.loc[index, 'Median_Income_Total'] = soup_level1.find('td', class_='count', id='medianHouseholdIncome').text        

        
    except Exception as e:
        print(e)

Beginning Data Retrieval
-----------------------------
Processing Record Armatage
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/armatage
Processing Record Audubon Park
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/audubon-park
Processing Record Bancroft
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/bancroft
Processing Record Cedar - Isles - Dean
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/cedar-isles-dean
Processing Record Cedar Riverside
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/cedar-riverside
Processing Record Central
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/central
Processing Record Como
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/como
Processing Record Cooper
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/cooper
Processing Record Corcoran
URL: http://www.mncompass.org/profiles/neighborhoods/minneapolis/corcoran
Processi

In [34]:
neighborhood_income_df.head()

,Neighborhood,Website_Link,IncomeLess35000_count,IncomeLess35000_percent,IncomeLess35to49_count,IncomeLess35to49_percent,IncomeLess50to74_count,IncomeLess50to74_percent,IncomeLess75to99_count,IncomeLess75to99_percent,Income100Plus_count,Income100Plus_percent,Median_Income_Total
0,Armatage,http://www.mncompass.org/profiles/neighborhood...,\n 296\n ...,\n 14.5%\n ...,\n 126\n ...,\n 6.2%\n ...,\n 253\n ...,\n 12.4%\n ...,\n 253\n ...,\n 253\n ...,"\n 1,117\n ...",\n 54.6%\n ...,"\n $ 125,329\n ..."
1,Audubon Park,http://www.mncompass.org/profiles/neighborhood...,\n 394\n ...,\n 17.7%\n ...,\n 319\n ...,\n 14.4%\n ...,\n 379\n ...,\n 17.0%\n ...,\n 365\n ...,\n 365\n ...,\n 768\n ...,\n 34.5%\n ...,"\n $ 67,343\n ..."
2,Bancroft,http://www.mncompass.org/profiles/neighborhood...,\n 279\n ...,\n 19.1%\n ...,\n 200\n ...,\n 13.7%\n ...,\n 317\n ...,\n 21.7%\n ...,\n 256\n ...,\n 256\n ...,\n 410\n ...,\n 28.0%\n ...,"\n $ 71,456\n ..."
3,Cedar - Isles - Dean,http://www.mncompass.org/profiles/neighborhood...,\n 238\n ...,\n 14.6%\n ...,\n 169\n ...,\n 10.3%\n ...,\n 207\n ...,\n 12.7%\n ...,\n 168\n ...,\n 168\n ...,\n 853\n ...,\n 52.2%\n ...,"\n $ 98,201\n ..."
4,Cedar Riverside,http://www.mncompass.org/profiles/neighborhood...,"\n 2,477\n ...",\n 73.0%\n ...,\n 403\n ...,\n 11.9%\n ...,\n 342\n ...,\n 10.1%\n ...,\n suppressed\n ...,\n suppressed\n ...,\n suppressed\n ...,\n,"\n $ 24,944\n ..."


In [36]:
#Save data to a csv file
output_data_file = "resources/MLPS_Hsld_Income_by_Neighborhood.csv"
output_path = os.path.join(output_data_file)
neighborhood_income_df.to_csv(output_path, header=True, index=False)